In [1]:
%pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=eff625d4b3cd8dfeb209cd1274253699b9d3e101593ddf6ba08912090c979df5
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [2]:
import wandb 
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="LAB8",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "RESNET",
    "dataset": "CIFAR-10",
    "OPTIMIZER":"adam",
    "epochs": 30,
    }
)

# simulate training
epochs = 30
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: sharma-87. Use `wandb login --relogin` to force relogin


acc,▁▁▇▅▆▆▇▇▆▇███████▇███▇████▇█
loss,█▄▂▄▂▂▁▂▂▂▂▁▁▂▁▂▁▂▁▁▁▁▁▁▂▁▁▁
acc,0.82037
loss,0.16728


In [24]:
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# def get_dataloader(is_train, batch_size, slice=5):
#     "Get a training dataloader"
#     full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
#     sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
#     loader = torch.utils.data.DataLoader(dataset=sub_dataset, 
#                                          batch_size=batch_size, 
#                                          shuffle=True if is_train else False, 
#                                          pin_memory=True, num_workers=2)
#     return loader

# def get_model(dropout):
#     "A simple model"
#     model = nn.Sequential(nn.Flatten(),
#                          nn.Linear(28*28, 256),
#                          nn.BatchNorm1d(256),
#                          nn.ReLU(),
#                          nn.Dropout(dropout),
#                          nn.Linear(256,10)).to(device)
#     return model

def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(5)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms



# Define the transform to apply to the data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the CIFAR-10 training set, only including the specified classes
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:13<00:00, 13012282.89it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
def filter(train_dataset) : 
      

      new_train_target=[]
      for label in train_dataset.targets:
        if(label==1):
          new_train_target.append(0)
        if(label==3):
          new_train_target.append(1)

        if(label==5):
          new_train_target.append(2)

        if(label==7):
         
          new_train_target.append(3)

        if(label==9):

          new_train_target.append(4)
          
        if(label==0):
          
          new_train_target.append(20)
        if(label==2):
         
          new_train_target.append(21)

        if(label==4):
         
          new_train_target.append(22)

        if(label==6):
        
          new_train_target.append(23)

        if(label==8):
         
          new_train_target.append(24)


        train_dataset.targets=new_train_target  

      return train_dataset


train_dataset = filter(trainset)
test_dataset = filter(testset)
def return_indices(train_set):

    indices=[]
    l=[0,1,2,3,4]
    for i in range(len(train_set.targets)):
      if(train_set.targets[i] in l ):
        indices.append(i)
    return indices 


data2=[]
target2=[]
indices=return_indices(train_dataset)
for index in indices:
  target2.append(train_dataset.targets[index])
  data2.append(train_dataset.data[index])

train_dataset.data=[]
train_dataset.target=[]  


train_dataset.data=data2
train_dataset.target=target2

import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img = self.data[idx]
        target = self.targets[idx]
        
        if self.transform:
            img = self.transform(img)
        
        return img, target


train_dataset = MyDataset(data=data2, targets=target2, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset , batch_size=100, shuffle=True)  

data3=[]
target3=[]
indices=return_indices(test_dataset)
for index in indices:
  target2.append(test_dataset.targets[index])
  data2.append(test_dataset.data[index])


test_dataset = MyDataset(data=data3, targets=target3, transform=transform)
test_loader = torch.utils.data.DataLoader(train_dataset , batch_size=100, shuffle=True)  

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

class ResNet18(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=False)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [39]:
import torch.nn.functional as F

class ResNet18_t(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet18_t, self).__init__()
        self.resnet = models.resnet18(pretrained=False)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = F.tanh(x)  # Replace relu with tanh
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [42]:
wandb.init(
        project="lab8_new",
        config={
            "epochs": 30,
            "batch_size": 128,
            "lr": 1e-3,
             "optimizer":"RMSPROP",
             "activation":"relu"
            })

In [43]:
wandb.config

{'epochs': 30, 'batch_size': 128, 'lr': 0.001, 'optimizer': 'RMSPROP', 'activation': 'relu'}

In [44]:
wandb.config.batch_size

128

In [46]:


# Copy your config 
config = wandb.config

# Get the data
train_dl = train_loader
valid_dl = test_loader
n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

# A simple MLP model
model = ResNet18().to(device)

# Make the loss and optimizer
loss_func = nn.CrossEntropyLoss()

optimizer = optim.RMSprop(model.parameters(), lr=config.lr)

# Training
example_ct = 0
step_ct = 0
for epoch in range(config.epochs):
  model.train()
  for step, (images, labels) in enumerate(train_dl):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      train_loss = loss_func(outputs, labels)
      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()
      
      example_ct += len(images)
      metrics = {"train/train_loss": train_loss, 
                "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch, 
                "train/example_ct": example_ct}
      
      if step + 1 < n_steps_per_epoch:
          # 🐝 Log train metrics to wandb 
          wandb.log(metrics)
          
      step_ct += 1

  val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))

  # 🐝 Log train and validation metrics to wandb
  val_metrics = {"val/val_loss": val_loss, 
                "val/val_accuracy": accuracy}
  wandb.log({**metrics, **val_metrics})
  
  print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

# If you had a test set, this is how you could log it as a Summary metric
wandb.summary['test_accuracy'] = 0.8

# 🐝 Close your wandb run 
wandb.finish()

Train Loss: 0.812, Valid Loss: 0.800783, Accuracy: 0.67
Train Loss: 0.646, Valid Loss: 0.647336, Accuracy: 0.74
Train Loss: 0.509, Valid Loss: 0.540008, Accuracy: 0.78
Train Loss: 0.536, Valid Loss: 0.439735, Accuracy: 0.83
Train Loss: 0.458, Valid Loss: 0.328407, Accuracy: 0.87
Train Loss: 0.338, Valid Loss: 0.374726, Accuracy: 0.86
Train Loss: 0.321, Valid Loss: 0.216728, Accuracy: 0.92
Train Loss: 0.256, Valid Loss: 0.189168, Accuracy: 0.93
Train Loss: 0.182, Valid Loss: 0.193720, Accuracy: 0.93
Train Loss: 0.254, Valid Loss: 0.116867, Accuracy: 0.96
Train Loss: 0.066, Valid Loss: 0.075782, Accuracy: 0.98
Train Loss: 0.133, Valid Loss: 0.086836, Accuracy: 0.97
Train Loss: 0.118, Valid Loss: 0.142020, Accuracy: 0.95
Train Loss: 0.081, Valid Loss: 0.070888, Accuracy: 0.97
Train Loss: 0.112, Valid Loss: 0.107312, Accuracy: 0.96
Train Loss: 0.057, Valid Loss: 0.064701, Accuracy: 0.98
Train Loss: 0.051, Valid Loss: 0.035242, Accuracy: 0.99
Train Loss: 0.044, Valid Loss: 0.035600, Accurac

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,███▆▄▄▃▃▃▃▂▂▂▁▃▂▂▁▁▁▂▁▁▁▁▁▁▂▁▂▁▁▂▁▁▁▂▂▁▁
val/val_accuracy,▁▃▃▄▅▅▆▇▇▇█▇▇█▇███████████████
val/val_loss,█▇▆▅▄▄▃▃▃▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,0.8
train/epoch,30.2766
train/example_ct,900000
train/train_loss,0.01275
val/val_accuracy,0.98927
val/val_loss,0.0295


In [23]:
wandb.finish()

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▆▆▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁
val/val_accuracy,▁▃▃▅▅▅▆▆▇▇▇▇█▇▇██▇███████████
val/val_loss,█▆▆▅▅▄▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
train/epoch,29.99574
train/example_ct,893400
train/train_loss,0.02442
val/val_accuracy,0.99297
val/val_loss,0.01879
